In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import math
import re
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

import warnings
warnings.filterwarnings('ignore')

In [67]:
def load_data(filename, values_dict = {}, train_data = False):
    df = np.asarray(pd.read_csv(filename, header=None, dtype=int))
    y = df[:,-1]
    
    df = df[:,0:df.shape[1]-1]
    x = np.zeros((df.shape[0],85))
            
    if values_dict == {}:
        for i in range(df.shape[1]):
            length = len(list(set(list(df[:,i]))))
            values_dict[i] = length
    
    for j in range(df.shape[0]):
        ohe_encoded = []
        for i in range(df.shape[1]):
            val = df[j][i]
            ohe_mat = np.zeros((values_dict[i]))
            ohe_mat[val-1] = 1
            ohe_encoded.extend(ohe_mat)
        ohe_encoded = np.asarray(ohe_encoded)
        x[j] = ohe_encoded

    if train_data == True:
        return x,y,values_dict
    else:
        return x,y

In [ ]:
PART = 'a'
BASE_DIR = '../'

train_path = os.path.join(BASE_DIR, 'data', 'Poker_Hand_dataset', 'poker-hand-training-true.data')
test_path  = os.path.join(BASE_DIR, 'data', 'Poker_Hand_dataset', 'poker-hand-testing.data')

# if PART == 'a':

Xtrain, Ytrain, values_dict = load_data(train_path,train_data=True)
Xtest, Ytest = load_data(test_path, values_dict)

In [86]:
BATCH_SIZE = 1024
NUMBER_OF_INPUTS = 85
HIDDEN_LAYERS_UNITS = [10]
NUMBER_OF_OUTPUTS = 2
LEARNING_RATE = 0.1

In [161]:
class NN_architecture:
    def __init__(self, learning_rate, batch_size, num_of_inputs, hidden_layer_units, num_of_outputs):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_of_inputs = num_of_inputs
        self.hidden_layer_list = hidden_layer_units
        self.num_of_outputs = num_of_outputs
    
    def sigmoid_activation(self,data):
        return (1.0/1.0 + np.exp(-data))
    
    def relu_activation(self,data):
        return np.multiply(data>0,data)
    
    def sigmoid_der(self,data):
        x = self.sigmoid_activation(data)
        return np.multiply(x,1.0-x)
    
    def relu_der(self,data):
        temp = np.ones(data.shape,dtype=float)
        return np.multiply(temp>0,temp)
                    
    def initialize(self):
        neuron_count = [self.num_of_inputs]
        neuron_count.extend(self.hidden_layer_list)
        neuron_count.append(self.num_of_outputs)
        self.neuron_count = neuron_count
        params = {}
        np.random.seed(2189)
        
        # xavier initialization
        for i in range(1, len(neuron_count)):
            params["W" + str(i)] = np.random.normal(0,1,(neuron_count[i],neuron_count[i-1]))*np.sqrt(2.0/neuron_count[i-1])
            params["b" + str(i)] = np.zeros((neuron_count[i],1),dtype=float)
        
        self.params = params
        self.num_of_layers = len(neuron_count)        
        return
    
    def forward_propagation(self, X, activation_function):        
        forward_prop = {}
        data = (X.T).copy()
        forward_prop["a0"] = data # n*m

        for i in range(1,self.num_of_layers-1):
            data = np.add(np.dot(self.params["W"+str(i)], data),self.params["b"+str(i)])
            forward_prop["z"+str(i)] = data

            if activation_function == "relu":
                data = self.relu_activation(data)
            elif activation_function == "sigmoid":
                data = self.sigmoid_activation(data)

            forward_prop["a"+str(i)] = data
        
        data = np.dot(self.params["W"+str(self.num_of_layers-1)], data) + self.params["b"+str(self.num_of_layers-1)]
        forward_prop["z"+str(self.num_of_layers-1)] = data
        data = self.sigmoid_activation(data)
        forward_prop["a"+str(self.num_of_layers-1)] = data
        
        self.forward_prop = forward_prop
        return
    
    def backward_propagation(self, Y, activation_function):
        backward_prop = {}
        dataY = (Y).copy()
        backward_prop["dz"+str(self.num_of_layers-1)] = self.forward_prop["a" + str(self.num_of_layers-1)] - Y
        
        i = self.num_of_layers-2
        while i>=0:
            temp_mat = np.dot(self.params["W"+str(i+1)].T, backward_prop["dz" + str(i+1)])
            
            if activation_function == "sigmoid":
                temp_mat = np.multiply(temp_mat, self.sigmoid_der(self.forward_prop["a"+str(i)]))
            elif activation_function == "relu":
                temp_mat = np.multiply(temp_mat, self.relu_der(self.forward_prop["a"+str(i)]))
                
            backward_prop["dz" + str(i)] = temp_mat            
            i-=1
        self.backward_prop = backward_prop
        return    
    
    def update_params(self,Y):
        for i in range(1,self.num_of_layers):
            self.params["W"+str(i)] = self.params["W"+str(i)] - (self.learning_rate/Y.shape[0])*np.dot(self.backward_prop["dz"+str(i)],(self.forward_prop["a"+str(i-1)]).T)
            self.params["b"+str(i)] = self.params["b"+str(i)] - \
                                        ( (self.learning_rate/Y.shape[0])*np.sum(self.backward_prop["dz"+str(i)],axis=1) ).reshape((self.params["b"+str(i)].shape[0],1))
        return
    
    def predict(self,X,activation_function):
        data_x = (X.T).copy()
        for i in range(1,self.num_of_layers-1):
            data_x = np.dot(self.params["W"+str(i)],data_x) + self.params["b"+str(i)]
            if activation_function == "sigmoid":
                data_x = self.sigmoid_activation(data_x)
            elif activation_function == "relu":
                data_x = self.relu_activation(data_x)
        data_x = data_x.T
        data_x = np.exp(data_x)
        data_x = data_x/np.sum(data_x)
        return np.argmax(data_x,axis=1)
            
    
    def run(self,X,Y):
        self.initialize()
        for i in range(10):
            print(i)
            self.forward_propagation(X,'sigmoid')
            self.backward_propagation(Y,'sigmoid')
            self.update_params(Y)
        self.y_pred = self.predict(X,'sigmoid')
            

In [162]:
model = NN_architecture(LEARNING_RATE,BATCH_SIZE,NUMBER_OF_INPUTS,HIDDEN_LAYERS_UNITS,NUMBER_OF_OUTPUTS)
model.run(Xtrain,Ytrain)

0
1
2
3
4
5
6
7
8
9


In [163]:
print(model.neuron_count)

[85, 10, 2]


In [164]:
model.params["W2"].shape

(2, 10)

In [171]:
print(model.params["b1"])

[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
